In [1]:
import json
import pandas as pd
from datasets import load_from_disk
from transformers import AutoTokenizer

import torch
from tqdm import tqdm
import numpy as np
import random

random.seed(42)

'''data'''
data = json.load(open('../data/raw/wikipedia_documents.json'))
wiki = pd.DataFrame(data).T
dataset = load_from_disk("../data/raw/train_dataset/")
train_df = pd.DataFrame(dataset['train'])
valid_df = pd.DataFrame(dataset['validation'])
mrc = pd.concat([train_df, valid_df])

/opt/conda/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from sentence_transformers import SentenceTransformer
import torch

prompts = {
    "query": "query: ",        # 검색 쿼리 프롬프트
    "passage": "passage: "     # 문서 패시지 프롬프트
}

'''inference'''
model_name = "nlpai-lab/KoE5"
model = SentenceTransformer(
    model_name_or_path=model_name, 
    device='cuda', 
    similarity_fn_name='dot',
    truncate_dim=512,
    model_kwargs={"torch_dtype": torch.bfloat16},
    prompts=prompts,
    )
queries = mrc['question'].tolist()[:5]
wiki_list = wiki['text'].tolist()[:10]

In [6]:
len(queries), len(wiki_list)

(5, 10)

In [3]:
def sliding_window_tokenize(text, tokenizer, max_length, stride):
    tokens = tokenizer(
        text, truncation=True, max_length=max_length, stride=stride, return_overflowing_tokens=True
    )
    
    # 첫 번째 청크를 리스트로 추가
    chunks = [tokens["input_ids"]]
    
    # 초과된 토큰이 있을 경우 반복 처리
    while "overflowing_tokens" in tokens and len(tokens["overflowing_tokens"]) > 0:
        # 초과된 토큰을 기반으로 새로운 청크 생성
        tokens = tokenizer(
            tokenizer.decode(tokens["overflowing_tokens"]), 
            truncation=True, max_length=max_length, stride=stride, return_overflowing_tokens=True
        )
        chunks.append(tokens["input_ids"])  # 각 청크 추가
    
    # 각 청크를 개별적으로 decode하여 문자열로 변환 후 반환
    return [tokenizer.decode(chunk, skip_special_tokens=True) for chunk in chunks]


In [4]:
doc = wiki['text'].tolist()[0]

tokenizer = model.tokenizer
max_length = min(model.max_seq_length, model[0].auto_model.config.max_position_embeddings)  # 최대 시퀀스 길이
stride = int(max_length * 0.5)  # 50% 겹침

tokens = tokenizer(
    doc, truncation=True, max_length=max_length, stride=stride, return_overflowing_tokens=True
)

print("doc: ", doc)
print('tokens:')
print(tokens.keys())

doc:  이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.

이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.

# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.
# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다.

두 목록은 모두 가나다 순이다.

일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 "국가" 문서에서 설명하고 있다.
tokens:
dict_keys(['input_ids', 'attention_mask', 'overflow_to_sample_mapping'])


In [5]:
model.set_pooling_include_prompt(False)

In [19]:
# print(model.max_seq_length)
# print(model[0].auto_model.config.max_position_embeddings)

# print(model.get_max_seq_length())
# print(model.get_sentence_embedding_dimension())
# print(model.similarity_fn_name)

# print(model.tokenizer)

# if hasattr(model[0].auto_model.config, 'instructor'):
#     print("This is an INSTRUCTOR model.")
# else:
#     print("This is not an INSTRUCTOR model.")
for i,x in enumerate(model):
    print(i, x)

0 Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
1 Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': False})
2 Normalize()


In [3]:
# pool = model.start_multi_process_pool()
# encoded_query = model.encode_multi_process(
#     sentences=queries,
#     pool=pool,
#     show_progress_bar=True,
#     )
# encoded_wiki = model.encode_multi_process(
#     sentences=wiki_list, 
#     pool=pool,
#     show_progress_bar=True,
#     )

# model.stop_multi_process_pool(pool)

In [4]:
# print(len(queries))
# print(len(wiki_list))

encoded_query = model.encode(queries, prompt_name="query", show_progress_bar=True)
encoded_wiki = model.encode(wiki_list, prompt_name="passage", show_progress_bar=True)

In [5]:
model.eval()

print(len(queries))
print(len(wiki_list))

with torch.no_grad():
    encoded_query = model.encode(queries, prompt_name="query", show_progress_bar=True)
    encoded_wiki = model.encode(wiki_list, prompt_name="passage", show_progress_bar=True)

4192
60613


Batches:   0%|          | 0/131 [00:00<?, ?it/s]

Batches:  10%|▉         | 186/1895 [03:16<30:05,  1.06s/it]


KeyboardInterrupt: 

In [7]:
queries = queries[:1]
wiki_list = wiki_list[:10]

for i in range(len(queries)):
    query_embedding = model.encode(queries[i], prompt_name="web_search_query")

    # 배치로 document embedding을 처리
    all_scores = []
    for start_idx in tqdm(range(0, len(wiki_list), batch_size), desc=f"Processing query {i+1}/{len(queries)}"):
        print(f'start idx: {start_idx}')
        print(f'batch size: {batch_size}')
        print(f'wiki list len: {len(wiki_list)}')
        batch_wiki_list = wiki_list[start_idx:start_idx + batch_size]
        document_embeddings = model.encode(batch_wiki_list)
        scores = (query_embedding @ document_embeddings.T) * 100
        print(scores.shape)
        all_scores.append(scores)
    print(all_scores)

    # 점수 결합
    all_scores = np.concatenate(all_scores, axis=-1)
    print(all_scores)
    
    # 상위 top_k 문서 찾기
    top_indices = np.argsort(all_scores)[::-1][:top_k]
    print(top_indices)
    
    correct = False
    for idx in top_indices:
        print(f'idx: {idx}')
        print(f'wiki: {wiki_list[idx][:100]}')
        print(f'correct context: {correct_contexts[i][:100]}')
        if correct_contexts[idx] == wiki_list[idx]:  # 정답이 포함되어 있는지 확인
            correct = True
            correct_count += 1  # 정답이 포함되면 카운트 증가
            break
    
    
    
    
    break

Processing query 1/1:   0%|          | 0/2 [00:00<?, ?it/s]

start idx: 0
batch size: 5
wiki list len: 10


Processing query 1/1:  50%|█████     | 1/2 [00:05<00:05,  5.44s/it]

(5,)
start idx: 5
batch size: 5
wiki list len: 10


Processing query 1/1: 100%|██████████| 2/2 [00:09<00:00,  4.87s/it]

(5,)
[array([70.44482, 73.37528, 70.29869, 73.86983, 63.21244], dtype=float32), array([64.66734, 62.94172, 72.91665, 69.45655, 69.65785], dtype=float32)]
[70.44482 73.37528 70.29869 73.86983 63.21244 64.66734 62.94172 72.91665
 69.45655 69.65785]
[3 1 7 0 2]
idx: 3
wiki: 신라 지증왕 4년(503년)에 만들어진 것으로 추정되며, 1989년에 발견되었다. 발견 당시 신라시대 비석 중 현존하는 최고(最古)의 비석이었으나, 2009년 9월에 501년 혹은
correct context: 항일의병장인 이교문 (李敎文)의 손자이자 이일의 아들이다. 5대조 이유원이 보성군 문덕면 가내마을에 정착하였고 그의 아들들 중 이용순의 고조할아버지인 이기대(李箕大)는 저명한 성리
idx: 1
wiki: 원래 베트남 지역을 통치하던 쩐 왕조는 대대로 명나라에 공물을 바치는 속국이었다. 하지만 1400년에 쩐 왕조의 장군 호꾸이리가 반란을 일으켜 쩐 왕가를 대거 학살한 다음 제위에 
correct context: 항일의병장인 이교문 (李敎文)의 손자이자 이일의 아들이다. 5대조 이유원이 보성군 문덕면 가내마을에 정착하였고 그의 아들들 중 이용순의 고조할아버지인 이기대(李箕大)는 저명한 성리
idx: 7
wiki: 1600년(선조 33년) 의인왕후 박씨가 승하하자 왕비릉인 유릉(裕陵)의 터로 정해진 곳이다.

1608년(선조 41년) 선조가 승하하고 광해군이 즉위하면서 능을 건원릉의 서편에 
correct context: 항일의병장인 이교문 (李敎文)의 손자이자 이일의 아들이다. 5대조 이유원이 보성군 문덕면 가내마을에 정착하였고 그의 아들들 중 이용순의 고조할아버지인 이기대(李箕大)는 저명한 성리
idx: 0
wiki: 경주시 황남동 미추왕릉 지구에 있는 삼국시대 신라 무덤인 